In [86]:
# imports for loading environment variables
import os
from dotenv import load_dotenv
# imports for getting documents
import requests 
# imports for loading and searching documents
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
# imports for llm
from openai import OpenAI
import tiktoken
import logging
log = logging.getLogger(__name__)

### Question 1. Running Elastic

In [73]:
!curl localhost:9200

{
  "name" : "7e07e11bad27",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "XH2kx34LQ0SdD7Uf8XfFLg",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


##### The version.build_hash is 42f05b9372a9a4a470db3b52817899b99a76ee73

In [3]:
# Get the data
def get_documents(docs_url):
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    documents = []

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            documents.append(doc)
    return documents

In [4]:
# Load sample data
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
documents = get_documents(docs_url)
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Question 2. Indexing the data

In [15]:
# initialize elastic search client
es_client = Elasticsearch('http://localhost:9200') 
index_name = "course-question"

In [14]:
# Create index for our data
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

es_client.indices.create(index=index_name, body=index_settings)

In [73]:
# loading documents to elastic search
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:03<00:00, 252.31it/s]


##### We use index to add the data to elastic search

In [79]:
class Elasticsearch:
    def __init__(self,fields:list,boost:dict={},filter:dict={},type:str="best_fields"):
        self.fields = self.return_fields(fields,boost)
        if filter:
            self.filter = self.return_filter(filter)
        else:
            self.filter = filter
        self.type = type

    def return_fields(self,fields,boost_dict):
        """
        Return fields 
        """
        prepared_fields = []
        for field in fields:
            if field in boost_dict.keys():
                field+= '^'+str(boost_dict[field])
            prepared_fields.append(field)
        log.info(f"Fields: {prepared_fields}")
        return prepared_fields
    
    def return_filter(self,filter_dict):
        return {
            "term": filter_dict
            }
    
    def return_query_json(self,query):
        query_json = {
                "bool": {
                        "must": {
                            "multi_match": 
                                    {
                                    "query": query,
                                    "fields": self.fields,
                                    "type": self.type
                                    }
                                }
                        }
                }
        if self.filter:
            query_json['bool']['filter'] = self.filter
        return query_json

    def return_search_query(self,query,size):
        return {
                "size": size,
                "query": self.return_query_json(query)
               }
    
    def show_search_query(self):
        if self.search_query:
            log.info(self.search_query)
        else:
            log.error("Query not submitted yet")

    def search_documents(self,query,index_name,size=5,score=True):
        self.search_query = self.return_search_query(query,size)
        response = es_client.search(index=index_name, body=self.search_query)
        result_docs = []
        if score:
            for hit in response['hits']['hits']:
                result_docs.append([hit['_score'],hit['_source']])
        else:
            for hit in response['hits']['hits']:
                result_docs.append(hit['_source'])
        return result_docs
            

### Question 3. Searching

In [81]:
# Question 3
es = Elasticsearch(fields=['question','text'],
              boost={'question':4},
              )
query = 'How do I execute a command in a running docker container?'
index_name = "course-question"
es.search_documents(query,index_name)[0]


[84.050095,
 {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'}]

##### The score for the top ranking result is 84.05

### Question 4. Filtering

In [82]:
# Question 4
es = Elasticsearch(fields=['question','text'],
              boost={'question':4},
              filter={'course':'machine-learning-zoomcamp'}
              )
query = 'How do I execute a command in a running docker container?'
index_name = "course-question"
qn_4_results = es.search_documents(query,index_name,size=3,score=False)
qn_4_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

##### The 3rd question returned by the search engine is "How do I copy files from a different folder into docker container’s working directory?"

### Question 5. Building Prompt

In [83]:
def build_prompt(query, search_results):
    
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""
    
    for doc in search_results:
        context = context + f"""
                            Q: {doc['question']}
                            A: {doc['text']}
                             """.strip()
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [84]:
query = "How do I execute a command in a running docker container?"
prompt = build_prompt(query, qn_4_results)
prompt

'You\'re a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.\n    Use only the facts from the CONTEXT when answering the QUESTION.\n\n    QUESTION: How do I execute a command in a running docker container?\n\n    CONTEXT: \n    Q: How do I debug a docker container?\n                            A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)Q: How do I copy files from my local machine to docker container?\n                            A: You can copy files from your local machine into a Docker container using the docker cp command. Here\'s how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. 

In [85]:
print(f"The length of the prompt is {len(prompt)}")

The length of the prompt is 1559


#### Question 6. Tokens

In [87]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [89]:
def token_calculator(text):
    tokens = encoding.encode(prompt)
    return len(tokens)
ip_token_count = token_calculator(prompt)

##### Our prompt has 329 tokens

### Bonus: Generating the answer(ungraded)

In [90]:
# Load environment variables from the .env file
path = os.path.join('./','.env')
load_dotenv(path)

# Get the API key
api_key = os.getenv('OPENAI_API_KEY')

# Initialize openai client
client = OpenAI(api_key=api_key)

In [91]:

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [92]:
llm_result = llm(prompt)
llm_result

'To execute a command in a running Docker container, you can use the `docker exec` command. Here are the steps to do it:\n\n1. Find the container ID or name of the running container by using:\n   ```bash\n   docker ps\n   ```\n\n2. Once you have the container ID, you can execute a command inside the container with:\n   ```bash\n   docker exec -it <container-id> <command>\n   ```\n\nFor example, to start a bash session in the container, you would use:\n   ```bash\n   docker exec -it <container-id> bash\n   ```'

### Bonus: Calculating the costs (ungraded)

In [94]:
def cost_calculator(input_tokens,output_tokens,no_requests=1000):
    ip_cost_per_token = 0.005/1000
    op_cost_per_token = 0.015/1000
    total_cost = no_requests*(ip_cost_per_token*input_tokens)+(op_cost_per_token*output_tokens)
    return total_cost


In [95]:
input_tokens = ip_token_count
output_tokens = token_calculator(llm_result)
cost_calculator(input_tokens,output_tokens)

1.6499350000000002